# Deploying AI into production with FastAPI

## Chapter 1 - Introducion to FastAPI for Model Deployment

### Section 1.1 - GET and POST requests for AI

#### GET endpoint for model information

You're part of a machine learning team that has developed several machine learning models, each designed for different tasks such as sentiment analysis, product categorization, and customer churn prediction. You're working on deploying these models, and you need to create an endpoint that provides basic information about each model.

Your task is to implement a GET endpoint at route `/model-info/{model_id}` that retrieves and returns this essential model information.

In [1]:
%%writefile main.py
from fastapi import FastAPI, HTTPException

app = FastAPI()

# Add model_id as a path parameter in the route
@app.get("/model-info/{model_id}")
# Pass on the model id as an argument
async def get_model_info(model_id: int):
    # Check if the passed model id is 0
    if model_id == 0:
      	# Raise the right status code for not found
        raise HTTPException(status_code=404, detail="Model not found")
    model_info = get_model_details(id)  
    # Return the model id and info in the dict
    return {"model_id": model_id, "model_name": model_info}

Writing main.py


#### POST endpoint for model registration

While the GET endpoint you created earlier allows users to retrieve information about existing models, you now need a way for authorized team members to register new models or update information about existing ones.

You need to create a POST endpoint that allows team members to register new models or update existing ones. This endpoint will store model information on the server.

In [15]:
%%writefile main.py
from pydantic import BaseModel
from fastapi import FastAPI

app = FastAPI()

model_db = {}

class ModelInfo(BaseModel):
    model_id: int
    model_name: str
    description: str

# Specify the status code for successful POST request
@app.post("/register-model", status_code=201)
# Pass the model info from the request as function parameter 
def register_model(model_info: ModelInfo):
    # Add new model's information dictionary to the model database
    model_db[model_info.model_id] = model_info.model_dump()
    # Return model info dictionary corresponding to model along with success status code
    return {"message": "Model registered successfully", "model": model_info}, 201

Overwriting main.py


In [18]:
import requests

data = {
    "model_id":1, 
    "model_name": "cnn", 
    "description": "convolutional nn"}

url = "http://localhost:8000/register-model"
headers = {"Content_Type": "Application-json"}
response = requests.post(url, json=data, headers=headers)
print(response.json())

[{'message': 'Model registered successfully', 'model': {'model_id': 1, 'model_name': 'cnn', 'description': 'convolutional nn'}}, 201]


### Section 1.2 - FastAPI prediction with a pre-trained model

In [86]:
from datasets import load_dataset

ds = load_dataset("SIH/palmer-penguins")

In [87]:
df = ds['train'].to_pandas()
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,None,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [97]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

features = ['species', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g']

df_dataset = df[features]
df_dataset = df_dataset.dropna()
X = df_dataset.drop("species", axis=1)
y = df_dataset["species"]

# 1. Split first
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Perform cross-validation only on the training set
cv_scores = cross_val_score(pipe, X_train, y_train, cv=5)

# 3. See accuracy on each fold
print("Cross-validation accuracy scores:", cv_scores)
print("Mean CV accuracy:", np.mean(cv_scores))

# 4. Fit on train on full dataset
pipe.fit(X_train, y_train)

# 5. Predict and evaluate on the test set
y_pred = pipe.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

# Save the pipeline (includes scaler + model)
joblib.dump(pipe, "penguin_classifier.pkl")

# Load the saved pipeline
model = joblib.load("penguin_classifier.pkl")
print(type(model))
# y_pred = model.predict(X_test)

Cross-validation accuracy scores: [0.98181818 1.         1.         0.98148148 1.        ]
Mean CV accuracy: 0.9926599326599327
Accuracy: 0.9855072463768116
<class 'sklearn.pipeline.Pipeline'>


#### Load the pre-trained model

You're a data scientist at an animal conservation company. You've been given a pre-trained machine learning model that predicts penguin species.

Your task is to load this model so it can be used in an API. The model has been saved using `joblib`.

A pre-trained ML model is stored in the pickle file: `penguin_classifier.pkl`

Write a script to load the pickle file as a model. Test your script by running `python3 solution.py` in the terminal.

In [98]:
# Import the necessary module
import joblib

# Load the pre-trained model
model = joblib.load('penguin_classifier.pkl')

# Print the type of the loaded model
print(f"Loaded model type: {type(model)}")

Loaded model type: <class 'sklearn.pipeline.Pipeline'>


#### Create the prediction endpoint

In this exercise, you'll create a prediction endpoint that uses a pre-trained model to estimate diabetes progression.

The model has been trained on a dataset which has three features age, bmi and blood_pressure. It then predicts the diabetes progression score. Using these inputs, it predicts a diabetes progression score, which helps assess how the condition may develop over time.

You'll use FastAPI to create a POST endpoint that accepts patient data and returns a prediction of diabetes progression.

In [99]:
df_dataset.columns

Index(['species', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm',
       'body_mass_g'],
      dtype='object')

In [137]:
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd

class PengiunFeatures(BaseModel):
    bill_length_mm: float
    bill_depth_mm: float
    flipper_length_mm: float
    body_mass_g: float
    
# Create FastAPI instance
app = FastAPI()

# Create a POST request endpoint at the route "/predict"
def predict_progression(features: PengiunFeatures):
    input_data = pd.DataFrame([pengiun.model_dump()])
    
    print(input_data)
    # Use the predict method to make a prediction
    prediction = model.predict(input_data)
    return {"predicted_progression": prediction[0]}

pengiun = PengiunFeatures(bill_length_mm=39.1,
                          bill_depth_mm=18.7,
                          flipper_length_mm=181.0,
                          body_mass_g=3750.0)

print(pengiun.model_dump())
predict_progression(pengiun)

# # Create a POST request endpoint at the route "/predict"
# @app.post("/predict")
# async def predict_progression(features: PengiunFeatures):
#     input_data = [[
#         features.age,
#         features.bmi,
#         features.blood_pressure
#     ]]
    
#     # Use the predict method to make a prediction
#     prediction = model.predict(input_data)
#     return {"predicted_progression": float(prediction[0])}

{'bill_length_mm': 39.1, 'bill_depth_mm': 18.7, 'flipper_length_mm': 181.0, 'body_mass_g': 3750.0}
   bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g
0            39.1           18.7              181.0       3750.0


{'predicted_progression': 'Adelie'}

In [153]:
%%writefile main.py
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import joblib

# Load the pre-trained model
model = joblib.load('penguin_classifier.pkl')

# Print the type of the loaded model
print(f"Loaded model type: {type(model)}")
class PengiunFeatures(BaseModel):
    bill_length_mm: float
    bill_depth_mm: float
    flipper_length_mm: float
    body_mass_g: float
    
# Create FastAPI instance
app = FastAPI()

# # Create a POST request endpoint at the route "/predict"
@app.post("/predict")
async def predict_progression(features: PengiunFeatures):
    input_data = pd.DataFrame([features.model_dump()])
    
    prediction = model.predict(input_data)
    return {"predicted_progression": prediction[0]}


Overwriting main.py


In [156]:
pengiun = PengiunFeatures(bill_length_mm=39.1,
                          bill_depth_mm=18.7,
                          flipper_length_mm=181.0,
                          body_mass_g=3750.0)

url = "http://localhost:8000/predict"
data = pengiun.model_dump()
response = requests.post(url, json=data)
print(response.json())

{'predicted_progression': 'Adelie'}


#### Running the FastAPI app

Your FastAPI app has been saved in a Python file called `main.py`. You would like to run the app from a Python script using uvicorn.

To serve your FastAPI app directly via the Python script in `solution.py`, you need to finish adding the code block that sets up the host and port of the server where the API will run.

In [157]:
%%writefile solution.py
# Import the server module
import uvicorn
from main import app

if __name__ == "__main__":
    # Start the uvicorn server
    uvicorn.run(
	  app, 
      # Configure the host
      host="0.0.0.0",
      # Configure the port
      port=8080)

Writing solution.py


In [158]:
!python3 solution.py

Loaded model type: <class 'sklearn.pipeline.Pipeline'>
INFO:     Started server process [475095]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
INFO:     127.0.0.1:40836 - "GET /hello HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:40852 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:40852 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:40856 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:40848 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:40848 - "GET /openapi.json HTTP/1.1" 200 OK
^C
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [475095]


now post to the server on port 8080

In [161]:
pengiun = PengiunFeatures(bill_length_mm=39.1,
                          bill_depth_mm=18.7,
                          flipper_length_mm=181.0,
                          body_mass_g=3750.0)

url = "http://localhost:8080/predict"
data = pengiun.model_dump()
response = requests.post(url, json=data)
print(response.json())

{'predicted_progression': 'Adelie'}
